Writing a simple ARIMA (actually AR(2)) as a recurrence relation.

In [1]:
from IPython.display import display, Math, Latex
import sympy as sp

In [2]:
m, ytm1, ytm2 = sp.symbols("m y_{t-1} y_{t-2}")


In [3]:
eqn = m * ytm1 - ytm2

eqn

m*y_{t-1} - y_{t-2}

In [4]:
vtm1, vtm2 = sp.symbols("v_{t-1} v_{t-2}")

In [5]:
def step(y_tm1, y_tm2):
    # apply 
    y_t = eqn.subs({ytm1: y_tm1, ytm2: y_tm2}).expand()
    # factor
    t1 = y_t.subs({vtm1: 1, vtm2: 0})
    t2 = y_t.subs({vtm1: 0, vtm2: 1})
    poly = t1 * vtm1 + t2 * vtm2
    assert (y_t - poly).expand() == 0
    # display form
    latex_str = f'({sp.latex(t1)}) {sp.latex(vtm1)} - ({sp.latex(-t2)}) {sp.latex(vtm2)}'  # force term order
    # shift
    return poly, y_tm1, latex_str


In [6]:
# plug in in intial values
y_tm1, y_tm2 = vtm1, vtm2

(y_tm1, y_tm2)

(v_{t-1}, v_{t-2})

In [7]:
display(y_tm1)
for sv in range(10):
    y_tm1, y_tm2, latex_str = step(y_tm1, y_tm2)
    print(sv)
    print(latex_str)
    display(Math(latex_str))


v_{t-1}

0
(m) v_{t-1} - (1) v_{t-2}


<IPython.core.display.Math object>

1
(m^{2} - 1) v_{t-1} - (m) v_{t-2}


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

8
(m^{9} - 8 m^{7} + 21 m^{5} - 20 m^{3} + 5 m) v_{t-1} - (m^{8} - 7 m^{6} + 15 m^{4} - 10 m^{2} + 1) v_{t-2}


<IPython.core.display.Math object>

9
(m^{10} - 9 m^{8} + 28 m^{6} - 35 m^{4} + 15 m^{2} - 1) v_{t-1} - (m^{9} - 8 m^{7} + 21 m^{5} - 20 m^{3} + 5 m) v_{t-2}


<IPython.core.display.Math object>

In [8]:
p = 1
pm1 = 0

In [9]:
display(p)
for sv in range(10):
    p, pm1 = (m * p - pm1).expand(), p
    display(p)

1

m

m**2 - 1

m**3 - 2*m

m**4 - 3*m**2 + 1

...

m**6 - 5*m**4 + 6*m**2 - 1

m**7 - 6*m**5 + 10*m**3 - 4*m

m**8 - 7*m**6 + 15*m**4 - 10*m**2 + 1

m**9 - 8*m**7 + 21*m**5 - 20*m**3 + 5*m

m**10 - 9*m**8 + 28*m**6 - 35*m**4 + 15*m**2 - 1

In [10]:
# To solve.
# Look at recurrence:
#  r[2] = m * r[1] - r[0]
# look for solutions of the form r[k] = r**k
# (this is the characteristic function method, trying weaker powers in place of subscripts)
#  r**2 - m * r + 1 = 0
# solve, yielding:
#  r = m +- sqrt(m**2 - 4)/2 
# The claim is: these are the two r[k] = r**k solutions and all
# Pick a linear combination of these two solutions meeting initial conditions.
def poly(i: int):
    d = sp.sqrt(m*m - 4)
    return ((1/d) * ( ((m + d)/2)**i - ((m - d)/2)**i )).expand()

In [11]:
s = sp.Symbol('s')
poly_expr = ((1/sp.sqrt(m*m - 4)) * ( ((m + sp.sqrt(m*m - 4))/2)**s - ((m - sp.sqrt(m*m - 4))/2)**s ))

poly_expr

(-(m/2 - sqrt(m**2 - 4)/2)**s + (m/2 + sqrt(m**2 - 4)/2)**s)/sqrt(m**2 - 4)

In [12]:
display(poly_expr.subs({s: 0}).expand())

0

In [13]:
for sv in range(1, 12):
    display(poly_expr.subs({s: sv}).expand())

1

m

m**2 - 1

m**3 - 2*m

m**4 - 3*m**2 + 1

...

m**6 - 5*m**4 + 6*m**2 - 1

m**7 - 6*m**5 + 10*m**3 - 4*m

m**8 - 7*m**6 + 15*m**4 - 10*m**2 + 1

m**9 - 8*m**7 + 21*m**5 - 20*m**3 + 5*m

m**10 - 9*m**8 + 28*m**6 - 35*m**4 + 15*m**2 - 1